In [1]:
import os
import numpy as np
import h5py

import matplotlib.pyplot as plt

import ctypes
from ctypes import c_int, c_float, POINTER

from configs import root_dirs
from configs.tests import attention_test

from molnet.data import input_pipeline_online, utils
from molnet.data._c import bindings
from molnet.data.utils import get_image_and_atom_map_cpp

2024-11-27 16:15:19.382219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
clib = ctypes.CDLL(os.path.join("../molnet/data/_c", "peaks_lib.so"))

fp_p = POINTER(c_float)
int_p = POINTER(c_int)

# fmt: off
clib.peak_dist.argtypes = [
    c_int, c_int, c_int, c_int, fp_p,
    int_p, fp_p,
    fp_p, fp_p, c_float
] # fmt: on

def peak_dist(xyz, grid_dimensions, xyz_start, xyz_step, std):
#def peak_dist(atoms, n_xyz, xyz_start, xyz_step, std):
    nb = 1
    nx, ny, nz = grid_dimensions
    dist = np.empty([nb, nx, ny, nz], dtype=np.float32)
    dist_c = dist.ctypes.data_as(fp_p)
    N_atom = np.array(xyz.shape[0], dtype=np.int32).ctypes.data_as(int_p)
    pos = xyz[:, :3].astype(np.float32).ctypes.data_as(fp_p)
    xyz_start = np.array(xyz_start, dtype=np.float32).ctypes.data_as(fp_p)
    xyz_step = np.array(xyz_step, dtype=np.float32).ctypes.data_as(fp_p)

    # fmt: off
    clib.peak_dist(
        nb, nx, ny, nz, dist_c,
        N_atom, pos,
        xyz_start, xyz_step, std
    ) # fmt: on

    return dist


In [3]:
xyz = np.random.uniform(0, 16, (16, 3))
xyz[:, 2] /= 8 
grid_dimensions = (128, 128, 20)
xyz_start = [0, 0, 0]
xyz_step = [0.125, 0.125, 0.1]
std = 0.2

In [4]:
%timeit dist = peak_dist(xyz, grid_dimensions, xyz_start, xyz_step, std)

5.01 ms ± 11.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
print(dist.shape)

In [10]:
def get_target(xyz, sw):
    return utils.compute_atom_maps(
        xyz, sw,
        z_cutoff=2.0,
        map_resolution=0.128,
        sigma=0.2
    )

In [11]:
%timeit y = get_target(batch["xyz"][0], batch["sw"][0])
y = get_target(batch["xyz"][0], batch["sw"][0])
print(y.shape)

70.7 ms ± 31.9 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(5, 125, 125, 20)


In [12]:
def get_one_map(xyz, sw):
    z_max = np.max(xyz[:, 2])
    return utils._create_all_atom_position_maps_cpp(
        [xyz], sw,
        z_max=z_max,
        sw_size=16.0,
        z_cutoff=2.0,
        map_resolution=0.125,
        sigma=0.2
    )

In [13]:
%timeit y = get_one_map(batch["xyz"][0], batch["sw"][0])
y = get_one_map(batch["xyz"][0], batch["sw"][0])
print(y.shape)

14.9 ms ± 28.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
(1, 128, 128, 20)


In [16]:
def get_target_and_stack(xyz, sw):
    return np.stack(
        [
            utils.compute_atom_maps(
                xyz_i, sw_i,
                z_cutoff=2.0,
                map_resolution=0.128,
                sigma=0.2
            ) for xyz_i, sw_i in zip(xyz, sw)
        ]
    )

In [17]:
%timeit y = get_target_and_stack(batch["xyz"], batch["sw"])

1.17 s ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
config = attention_test.get_config()
config.root_dir = root_dirs.get_root_dir()
config.batch_size = 12

datasets = input_pipeline_online.get_datasets(config)

batch = next(datasets["train"])

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 3756706523579900210
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 3756706523579900210
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing te

In [18]:
def get_peak_dist_species(batch):
    return bindings.peak_dist_species(
        batch["xyz"],
        (128, 128, 10),
        [0, 0, -1],
        [0.125, 0.125, 0.1],
        0.2
    )

%timeit y = get_peak_dist_species(batch)
y = get_peak_dist_species(batch)
print(y.shape)

124 ms ± 824 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
(12, 5, 128, 128, 10)
